### IMPORTS

In [1]:
import torch
from torch import nn
from torch import functional as F
import matplotlib.pyplot as plt
import numpy as np
from livelossplot import PlotLosses
from Tools.NNtools import *
import math
from torch.distributions.multivariate_normal import MultivariateNormal


In [2]:
# Find what device to work with
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
def identity(m):
    return m

In [4]:
class parallel_MLP(nn.Module):
            def __init__(self, layerwidth,nb_layers=1):
                super(parallel_MLP, self).__init__()
                self.layerwidth = layerwidth
                self.activation=nn.Tanh()# use identity for debugging
                self.requires_grad_(False)
                self.nb_layers=nb_layers
                self.param_count=2*layerwidth+(nb_layers-1)*(layerwidth**2+layerwidth)+layerwidth+1
                self.splitting=[layerwidth]*2+[layerwidth**2,layerwidth]*(nb_layers-1)+[layerwidth,1]
                
            def forward(self,theta,x):
                nb_theta=theta.shape[0]
                nb_x=x.shape[0]
                theta=theta.split(self.splitting,dim=1)
                input_x=x.view(nb_x,1,1)
                print(input_x.shape)
                m=torch.matmul(theta[0].view(nb_theta,1,self.layerwidth,1),input_x)
                print(m.shape)
                m=m.add(theta[1].reshape(nb_theta,1,self.layerwidth,1))
                print(m.shape)
                m=self.activation(m)
                print(m.shape)
                for i in range(self.nb_layers-1):
                    print(m.shape)                   
                    m=torch.matmul(theta[2*i+2].view(-1,1,self.layerwidth,self.layerwidth),m)
                    print(m.shape)
                    m=m.add(theta[2*i+3].reshape(-1,1,self.layerwidth,1))
                    print(m.shape)
                    m=self.activation(m)
                    print(m.shape)
                m=torch.matmul(theta[2*(self.nb_layers-1)+2].view(nb_theta,1,1,self.layerwidth),m)
                print(m.shape)
                m=m.add(theta[2*(self.nb_layers-1)+3].reshape(nb_theta,1,1,1))
                print(m.shape)
                return m.squeeze(-1)


In [5]:
model=parallel_MLP(10,2)

In [6]:

model.splitting

[10, 10, 100, 10, 10, 1]

In [7]:
param_count=model.param_count
param_count

141

In [8]:
x=torch.rand(1,1)

theta=torch.rand(1,param_count)

model(theta,x)

torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])


tensor([[[5.0836]]])

In [9]:
x=torch.rand(1,1)

theta=torch.rand(1,param_count)
theta_=torch.rand(1,param_count)

y=model(theta,x)
y_=model(theta_,x)
y_m=torch.cat([y,y_],dim=0)
y_M=model(torch.stack([theta.squeeze(),theta_.squeeze()]),x)
#print(torch.stack([y_m,y_M],dim=2))
torch.eq(y_m,y_M)
#y_m,y_M

torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 10, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 10, 1])
torch.Size([2, 1, 1, 1])
torch.Size([2, 1, 1, 1])


tensor([[[True]],

        [[True]]])

In [10]:
x=torch.rand(3,1)
x_=torch.rand(5,1)
theta=torch.rand(1,param_count)

y=model(theta,x)
y_=model(theta,x_)
y_m=torch.cat([y,y_],dim=1)
y_M=model(theta,torch.cat([x,x_],dim=0))
#print(torch.stack([y_m,y_M],dim=2))
print(y_m==y_M)

torch.Size([3, 1, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 10, 1])
torch.Size([1, 3, 1, 1])
torch.Size([1, 3, 1, 1])
torch.Size([5, 1, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 10, 1])
torch.Size([1, 5, 1, 1])
torch.Size([1, 5, 1, 1])
torch.Size([8, 1, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 10, 1])
torch.Size([1, 8, 1, 1])
torch.Size([1, 8, 1, 1])
tensor([[[True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True]]])


In [11]:
x=torch.rand(200,1)
theta=torch.rand(100,param_count)

y=model(theta,x)

torch.Size([200, 1, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 1, 1])
torch.Size([100, 200, 1, 1])


In [12]:
def get_logprior_fn(device):
    S = torch.eye(param_count).to(device)
    mu = torch.zeros(param_count).to(device)
    prior = MultivariateNormal(mu, scale_tril=S)
    def logprior(x):
        v = prior.log_prob(x).unsqueeze(-1)
        return v
    return logprior

def _log_norm(x, mu, std):
    return -0.5 * torch.log(2*np.pi*std**2) -(0.5 * (1/(std**2))* (x-mu)**2)

# added parallelized versions
def get_loglikelihood_parallel_fn(device):
    def loglikelihood_parallel(theta, model, x, y, sigma_noise):
        y_pred = model(theta,x)
        L = _log_norm(y_pred, y, torch.tensor([sigma_noise],device=device))
        return torch.sum(L,1)
    return loglikelihood_parallel

# added parallelized versions
def get_logposterior_parallel_fn(device):
    logprior = get_logprior_fn(device)
    loglikelihood_parallel = get_loglikelihood_parallel_fn(device)
    def logposterior_parallel(theta, model, x, y, sigma_noise):
        return logprior(theta).add(loglikelihood_parallel(theta, model, x, y, sigma_noise))
    return logposterior_parallel

loglikelihood_parallel=get_loglikelihood_parallel_fn('cpu')
logposterior_parallel=get_logposterior_parallel_fn('cpu')

In [13]:
x=torch.rand(200,1)
y=torch.rand(200,1)


In [15]:
theta=torch.rand(10,param_count)
logposterior_parallel(theta,model,x,y,0.1).shape

torch.Size([200, 1, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 10, 1])
torch.Size([10, 200, 1, 1])
torch.Size([10, 200, 1, 1])


torch.Size([10, 1])

In [16]:
theta=torch.rand(100,param_count)
logposterior_parallel(theta,model,x,y,0.1).shape

torch.Size([200, 1, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 10, 1])
torch.Size([100, 200, 1, 1])
torch.Size([100, 200, 1, 1])


torch.Size([100, 1])